### 1. Import Libraries

In [16]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import os

### 2. Data Obtention

In [17]:
tickers = yf.Tickers('ASML NEE PH SBGSF')
hist = tickers.history(start='2021-01-01',end='2024-12-31')
print('Data:')
print(hist)

[*********************100%***********************]  4 of 4 completed

Data:
Price            Close                                    Dividends            \
Ticker            ASML        NEE          PH       SBGSF      ASML  NEE   PH   
Date                                                                            
2021-01-04  482.239594  67.349091  250.678802  137.562943       0.0  0.0  0.0   
2021-01-05  487.409241  67.848175  256.332855  136.498001       0.0  0.0  0.0   
2021-01-06  472.807007  71.169342  264.653595  142.646896       0.0  0.0  0.0   
2021-01-07  483.618835  71.949738  265.134125  143.424774       0.0  0.0  0.0   
2021-01-08  490.669159  73.619408  264.474579  143.721085       0.0  0.0  0.0   
...                ...        ...         ...         ...       ...  ...  ...   
2024-12-23  721.039978  72.489998  644.128357  247.080002       0.0  0.0  0.0   
2024-12-24  719.710022  72.910004  651.031982  248.800003       0.0  0.0  0.0   
2024-12-26  715.859985  72.370003  649.884705  256.000000       0.0  0.0  0.0   
2024-12-27  713.590027

In [18]:
adj_close = hist['Close']
print('Adj Close:')
print(adj_close)

Adj Close:
Ticker            ASML        NEE          PH       SBGSF
Date                                                     
2021-01-04  482.239594  67.349091  250.678802  137.562943
2021-01-05  487.409241  67.848175  256.332855  136.498001
2021-01-06  472.807007  71.169342  264.653595  142.646896
2021-01-07  483.618835  71.949738  265.134125  143.424774
2021-01-08  490.669159  73.619408  264.474579  143.721085
...                ...        ...         ...         ...
2024-12-23  721.039978  72.489998  644.128357  247.080002
2024-12-24  719.710022  72.910004  651.031982  248.800003
2024-12-26  715.859985  72.370003  649.884705  256.000000
2024-12-27  713.590027  72.110001  642.292664  252.000000
2024-12-30  696.150024  71.760002  635.319153  248.500000

[1004 rows x 4 columns]


In [19]:
if os.path.exists('data.xlsx'):
    os.remove('data.xlsx')
adj_close.to_excel('data.xlsx')

### 3. Portfolio Analysis

In [20]:
adj_close_values = adj_close.values
print('Adj Close values:')
print(np.round(adj_close_values,4))
print(adj_close_values.shape)

Adj Close values:
[[482.2396  67.3491 250.6788 137.5629]
 [487.4092  67.8482 256.3329 136.498 ]
 [472.807   71.1693 264.6536 142.6469]
 ...
 [715.86    72.37   649.8847 256.    ]
 [713.59    72.11   642.2927 252.    ]
 [696.15    71.76   635.3192 248.5   ]]
(1004, 4)


In [21]:
R = np.log(adj_close_values[1:] / adj_close_values[:-1])
print('R:')
print(np.round(R*100, 4), '%')

R:
[[ 1.0663  0.7383  2.2304 -0.7772]
 [-3.0417  4.779   3.1945  4.4062]
 [ 2.261   1.0906  0.1814  0.5438]
 ...
 [-0.5364 -0.7434 -0.1764  2.8528]
 [-0.3176 -0.3599 -1.1751 -1.5748]
 [-2.4743 -0.4865 -1.0917 -1.3986]] %


In [22]:
RE = np.mean(R, axis=0)*252
RI = np.std(R, axis=0)*np.sqrt(252)
print('RE:', np.round(RE*100, 4), '%')
print('RI:', np.round(RI*100, 4), '%')

RE: [ 9.2239  1.5939 23.3648 14.8577] %
RI: [42.184  26.9508 27.6346 34.2982] %


In [23]:
Sharpes = RE / RI
print('Sharpes:', np.round(Sharpes, 4))

Sharpes: [0.2187 0.0591 0.8455 0.4332]


In [24]:
S = np.cov(R, rowvar=False)
print('S:')
print(S*100, '%')

S:
[[0.07068503 0.0102608  0.0226113  0.0233615 ]
 [0.0102608  0.02885193 0.00673317 0.00556545]
 [0.0226113  0.00673317 0.03033476 0.01366003]
 [0.0233615  0.00556545 0.01366003 0.04672781]] %


In [25]:
corr = np.corrcoef(R, rowvar=False)
print('Correlation:')
print(corr)

Correlation:
[[1.         0.22721146 0.4883054  0.40648946]
 [0.22721146 1.         0.2275947  0.15157408]
 [0.4883054  0.2275947  1.         0.362822  ]
 [0.40648946 0.15157408 0.362822   1.        ]]


In [26]:
n_assets = R.shape[1]
weights = np.ones(n_assets) / n_assets
print('Weights:', np.round(weights, 4))
print('Sum of weights:', np.sum(weights))

Weights: [0.25 0.25 0.25 0.25]
Sum of weights: 1.0


In [27]:
def rep(w,r):
    return w @ np.transpose(r)

def varp(w,s):
    return w @ s @ np.transpose(w)

In [28]:
ReP = rep(weights, RE)
varP = varp(weights, S)
RiP = np.sqrt(varP)*np.sqrt(252)
SharpeP = ReP / RiP
print('Portafolio pre-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio pre-optimización:
ReP: 12.2601 %
varP: 0.0002
RiP: 23.1743 %
SharpeP: 0.529


### 4. Portfolio Optimization

In [29]:
def constr(w):
    return np.sum(w) - 1

num_assets = 4
bounds = [(0, None)] * num_assets

constraints = [{'type': 'eq', 'fun': constr}]

res = minimize(
    fun=lambda w: varp(w, S),
    x0=weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': True, 'maxiter': 1000, 'ftol': 1e-12}
)

print("Éxito:", res.success)
print("Mensaje:", res.message)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00016461493255230168
            Iterations: 19
            Function evaluations: 95
            Gradient evaluations: 19
Éxito: True
Mensaje: Optimization terminated successfully


In [30]:
ReP = rep(res.x, RE)
varP = varp(res.x, S)
RiP = np.sqrt(varP)*np.sqrt(252)
SharpeP = ReP / RiP
print('Portafolio post-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio post-optimización:
ReP: 11.9074 %
varP: 0.0002
RiP: 20.3674 %
SharpeP: 0.5846
